<h1 style="background-color:#FF7733;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
📐 תהליך הסבת שכבת חזיתות בתהליך לבנק"ל מודרני 📐 

In [ ]:
print('13. Fronts In-Process:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess

In [ ]:
print('  13.a Reading inputs')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
Results_gdb = r"\\mapi_shares\MNCDB\צוות מידע\תוצרים\Results.gdb"

<h1 style="text-align:right;font-size:100%">
כדי לשחזר גיאומטריה מקורית בסיום הסבה (TEMP_ID) הוספת שדה של מספר מזהה זמני לחלקות בתהליך

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Fronts_InProcessp-pre_calc.bat"]);

<h1 style="text-align:right;font-size:100%">
שכבת חזיתות בתהליך קיימת

In [ ]:
fronts_inprocess_cols = ['TEMP_ID', 'FRONT_LEGAL_LENGTH', 'CUR_RADIUS', 'ENTITY_STATE']

fronts_inprocess = gpd.read_file(Results_gdb,
                                 layer = 'CADSDE_Lines_InProcess',
                                 include_fields = fronts_inprocess_cols,
                                 ignore_geometry = True)

del fronts_inprocess_cols

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  13.b Cleaning data')

In [ ]:
print('  13.c Data conversion')

<h1 style="text-align:right;font-size:100%">
חזיתות בעלות שטח רשום ריק יקבלו את הערך 0
<h1 style="text-align:right;font-size:100%">
 חזיתות בעלות רדיוס ריק יקבלו את הערך 0

In [ ]:
fronts_inprocess['FRONT_LEGAL_LENGTH'] = fronts_inprocess['FRONT_LEGAL_LENGTH'].fillna(0.0)
fronts_inprocess['CUR_RADIUS'] = fronts_inprocess['CUR_RADIUS'].fillna(0.0)


fronts_inprocess.rename(columns = {'FRONT_LEGAL_LENGTH' : 'LegalLength',
                                   'CUR_RADIUS'         : 'Radius'},
                        inplace=True)

<h1 style="text-align:right;font-size:100%">
סוג החזית יקבע לפי קיום של רדיוס

In [ ]:
fronts_inprocess['LineType'] = np.where(fronts_inprocess['Radius'] == 0.0, 1, 2)

<h1 style="text-align:right;font-size:100%">
ENTITY_STATE סטאטוס החזית תקבע לפי שדה

In [ ]:
fronts_inprocess['LineStatus'] = np.nan
fronts_inprocess.loc[fronts_inprocess['ENTITY_STATE'] == 0, 'LineStatus'] = 1
fronts_inprocess.loc[fronts_inprocess['ENTITY_STATE'] == 1, 'LineStatus'] = 2

fronts_inprocess.drop(columns='ENTITY_STATE', inplace=True)

In [ ]:
fronts_inprocess['FrontUniqueID'] = range(1, 1 + len(fronts_inprocess))


fronts_inprocess['StartPointUniqueID'] = 0


fronts_inprocess['EndPointUniqueID'] = 0


fronts_inprocess['LeftParcelUniqueID'] = 0


fronts_inprocess['RightParcelUniqueID'] = 0

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
 print('  13.d Defining new Layer')

<h1 style="text-align:right;font-size:100%">
הגדרת סוגי שדות נתונים

In [ ]:
fronts_inprocess = fronts_inprocess.astype({'FrontUniqueID'       : int,
                                            'LineType'            : int,
                                            'LineStatus'          : 'Int64',
                                            'Radius'              : float,
                                            'LegalLength'         : float,
                                            'StartPointUniqueID'  : int,
                                            'EndPointUniqueID'    : int,
                                            'LeftParcelUniqueID'  : int,
                                            'RightParcelUniqueID' : int,
                                            'TEMP_ID'             : int})

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  13.e Exporting results to product folder')

In [ ]:
csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\InProcessFronts_data.csv'

fronts_inprocess.to_csv(csv_dir, index=False)

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Fronts In-Process-Save as Feature Class.bat"]);